In [ ]:
from tweepy import API
from tweepy import Cursor
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import json
import io
import os
import datetime
import time
import twitter_credentials

count=0
retweet_count = 0
lang_count = 0
class TwitterClient():
    def __init__(self,twitter_user=None):
        self.auth = TwitterAuthenticator().authenticate_twitter_app()
        self.twitter_client = API(self.auth)
        self.twitter_user = twitter_user
        
    def get_user_timeline_tweets(self, num_tweets):
        tweets = []
        global count,retweet_count,lang_count
        for tweet in Cursor(self.twitter_client.user_timeline, id= self.twitter_user).items(num_tweets):
#             print("the tweet language is ",tweet.lang)
#             if tweet.lang == 'pt':
#                 lang_count+=1
            print(tweet.id)
            print(tweet.created_at)
            x = datetime.datetime(2019, 9, 10)
            y= datetime.datetime(2019, 9, 5)
            
            if tweet.created_at>x:
                continue
            if tweet.created_at<y:
                break
            if hasattr(tweet, 'retweeted_status'):
                continue
                if retweet_count > 330:
                    continue
                retweet_count = retweet_count+1
#                 print("the "+str(count)+" is retweet and the retweet count is ",retweet_count)
                
             
            tweets = tweets+[tweet]           
            count = count +1
            os.system('twarc replies '+str(tweet.id)+' > '+self.twitter_user+"_"+str(count)+'.json')
            time.sleep(300)
#             if(.15*count < retweet_count):
#                 tweets = TwitterClient().get_user_timeline_tweets(count+15)
#             with open('data.txt','a') as json_file:
#                 data = json.load(tweet, ensure_ascii=False, indent=4)
            
        return tweets
    
    def get_friend_timeline_tweets(self, num_tweets):
        tweets = []
        for tweet in Cursor(self.twitter_client.friends, id= self.twitter_user).items(num_tweets):
            tweets = tweets+[tweet]
       
            
        return tweets
    
    def get_home_timeline_tweets(self, num_tweets):
        tweets = []
        for tweet in Cursor(self.twitter_client.home_timeline, id= self.twitter_user).items(num_tweets):
            tweets = tweets+[tweet]
       
        return tweets
        
class TwitterAuthenticator():
    def authenticate_twitter_app(self):
        auth = OAuthHandler(twitter_credentials.CONSUMER_KEY,twitter_credentials.CONSUMER_SECRET)
        auth.set_access_token(twitter_credentials.ACCESS_TOKEN,twitter_credentials.ACCESS_TOKEN_SECRET)
        return auth
        

class TwitterStramer():
    
    def __init__(self):
        self.twitter_authenticator = TwitterAuthenticator()
    
    def stream_tweets(self, fetched_tweets_filename,hash_tag_list):
        listener = TwitterListener(fetched_tweets_filename)
        auth = self.twitter_authenticator.authenticate_twitter_app()
        stream = Stream(auth, listener)
        stream.filter(track=hash_tag_list)
        

class TwitterListener(StreamListener):
    
    def __init__(self, fetched_tweets_filename):
        self.fetched_tweets_filename = fetched_tweets_filename
        
    def on_data(self, data):
        try:
#             print(data)
            with open(self, fetched_tweets_filename,'a') as tf:
                tf.write(data)
            return True
        except BaseException as e:
            print("Error on data "+ str(e))
        return True
    def on_error(self, status):
        if status == 420:
            return False # Returning false and teriminate prrogram in case the rate limit exceeds
        print(status)
        
if __name__ == "__main__":
    hash_tag_list = ["donald trump","hillary clinton"]
    fetched_tweets_filename = "tweets.json"
    
    twitter_user_list = ['AmitShah','myogiadityanath','rashtrapatibhvn','RahulGandhi','narendramodi','jairbolsonaro','LulaOficial','FlavioBolsonaro','CarlosBolsonaro','BolsonaroSP']
    
    global count,retweet_count,lang_count
    for twitter_user in twitter_user_list:
        print('-'*300)
        print("The user is ",twitter_user)
        count = 0
        retweet_count = 0
        twitter_client = TwitterClient(twitter_user)

        tweet_string = str(twitter_client.get_user_timeline_tweets(100))

        print('-'*300)

        print("The number of tweets is ",count)
        print("The number of Re-tweets is ",retweet_count)
        print("The Tweet percent ",0.15* count)  
    
#     print("The Hindi Language count is ",lang_count)
    
#     if (0.15*count)> retweet_count:
#         tf = io.open('tweet_exp.json','a+',encoding='utf8')
#         tf.write(tweet_string)
#         tf.close()
#     else:
#         tf = io.open('tweet_exp.json','w+',encoding='utf8')
#         tf.write(tweet_string)
#         tf.close()
    
    
#     print(twitter_client.get_user_timeline_tweets(1))
#     twitter_steamer = TwitterStramer()
#     twitter_steamer.stream_tweets(fetched_tweets_filename,hash_tag_list)